<a href="https://colab.research.google.com/github/tab-h/recipe_recs_knn/blob/main/knn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

In [4]:
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
import json
import re

In [5]:
ds = load_dataset("corbt/all-recipes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/451 [00:00<?, ?B/s]

(…)-00000-of-00004-237b1b1141fdcfa1.parquet:   0%|          | 0.00/128M [00:00<?, ?B/s]

(…)-00001-of-00004-d46654ac93566129.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

(…)-00002-of-00004-3b4f78b99eedadc2.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

(…)-00003-of-00004-2369b90eb0860a76.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2147248 [00:00<?, ? examples/s]

In [6]:
recipe_string = ds["train"][0]["input"]
recipe_string

'No-Bake Nut Cookies\n\nIngredients:\n- 1 c. firmly packed brown sugar\n- 1/2 c. evaporated milk\n- 1/2 tsp. vanilla\n- 1/2 c. broken nuts (pecans)\n- 2 Tbsp. butter or margarine\n- 3 1/2 c. bite size shredded rice biscuits\n\nDirections:\n- In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.\n- Stir over medium heat until mixture bubbles all over top.\n- Boil and stir 5 minutes more. Take off heat.\n- Stir in vanilla and cereal; mix well.\n- Using 2 teaspoons, drop and shape into 30 clusters on wax paper.\n- Let stand until firm, about 30 minutes.'

In [7]:
def parse_recipe(recipe_string):
    # regex to extract sections
    title_pattern = r"^(.*?)\n+Ingredients:"
    ingredients_pattern = r"Ingredients:\n(.*?)\n+Directions:"
    directions_pattern = r"Directions:\n(.*)"

    # extract title, ingredients, and directions using regex
    title_match = re.search(title_pattern, recipe_string, re.DOTALL)
    title = title_match.group(1) if title_match else "Untitled"

    ingredient_match = re.search(ingredients_pattern, recipe_string, re.DOTALL)
    direction_match = re.search(directions_pattern, recipe_string, re.DOTALL)

    # clean up the ingredients
    ingredients = []
    if ingredient_match:
        ingredient_list = ingredient_match.group(1).strip().split("\n")
        ingredients = [ingredient.strip("- ").strip() for ingredient in ingredient_list]

    # clean up and split directions into sentences
    directions = []
    if direction_match:
        direction_text = direction_match.group(1).strip()
        # split at periods, exclamation marks, or question marks, but keep the punctuation
        direction_sentences = re.split(r'(?<=[.!?])\s+', direction_text)
        directions = [sentence.strip("- ").strip() for sentence in direction_sentences]

    return title, ingredients, directions

In [8]:
# example input
t, i, d = parse_recipe(recipe_string)
print(t)
print(i)
print(d)

No-Bake Nut Cookies
['1 c. firmly packed brown sugar', '1/2 c. evaporated milk', '1/2 tsp. vanilla', '1/2 c. broken nuts (pecans)', '2 Tbsp. butter or margarine', '3 1/2 c. bite size shredded rice biscuits']
['In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.', 'Stir over medium heat until mixture bubbles all over top.', 'Boil and stir 5 minutes more.', 'Take off heat.', 'Stir in vanilla and cereal; mix well.', 'Using 2 teaspoons, drop and shape into 30 clusters on wax paper.', 'Let stand until firm, about 30 minutes.']


In [9]:
ds["train"][0]

{'input': 'No-Bake Nut Cookies\n\nIngredients:\n- 1 c. firmly packed brown sugar\n- 1/2 c. evaporated milk\n- 1/2 tsp. vanilla\n- 1/2 c. broken nuts (pecans)\n- 2 Tbsp. butter or margarine\n- 3 1/2 c. bite size shredded rice biscuits\n\nDirections:\n- In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.\n- Stir over medium heat until mixture bubbles all over top.\n- Boil and stir 5 minutes more. Take off heat.\n- Stir in vanilla and cereal; mix well.\n- Using 2 teaspoons, drop and shape into 30 clusters on wax paper.\n- Let stand until firm, about 30 minutes.'}

In [10]:
titles_list = []
ingredients_list = []
directions_list = []

count = 0
for recipe_str in ds["train"]:
    recipe_str = recipe_str["input"]
    title, ingredients, directions = parse_recipe(recipe_str)
    titles_list.append(title)
    ingredients_list.append(ingredients)
    directions_list.append(directions)

    count += 1

    if count % 100000 == 0:
        print("Recipes parsed: ", count)

Recipes parsed:  100000
Recipes parsed:  200000
Recipes parsed:  300000
Recipes parsed:  400000
Recipes parsed:  500000
Recipes parsed:  600000
Recipes parsed:  700000
Recipes parsed:  800000
Recipes parsed:  900000
Recipes parsed:  1000000
Recipes parsed:  1100000
Recipes parsed:  1200000
Recipes parsed:  1300000
Recipes parsed:  1400000
Recipes parsed:  1500000
Recipes parsed:  1600000
Recipes parsed:  1700000
Recipes parsed:  1800000
Recipes parsed:  1900000
Recipes parsed:  2000000
Recipes parsed:  2100000


In [11]:
titles_list[0:10]

['No-Bake Nut Cookies',
 "Jewell Ball'S Chicken",
 'Creamy Corn',
 'Chicken Funny',
 'Reeses Cups(Candy)  ',
 'Cheeseburger Potato Soup',
 'Rhubarb Coffee Cake',
 'Scalloped Corn',
 "Nolan'S Pepper Steak",
 'Millionaire Pie']

In [12]:
recipe_dict = {
    "titles": titles_list,
    "ingredients": ingredients_list,
    "directions": directions_list
}

recipe_df = pd.DataFrame(recipe_dict)

In [13]:
print(recipe_df.isna().sum())

titles         0
ingredients    0
directions     0
dtype: int64


In [14]:
# split into train+validation and test (test = 20% of total data)
train_valid, test_dataset = train_test_split(recipe_df, test_size=0.2, random_state=42)

# split train+valid into train and validation (valid = 25% of 80% = 20% of total data)
train_dataset, validation_dataset = train_test_split(train_valid, test_size=0.25, random_state=42)

print("Training dataset size: ", len(train_dataset))
print("Validation dataset size: ", len(validation_dataset))
print("Test dataset size: ", len(test_dataset))

Training dataset size:  1288348
Validation dataset size:  429450
Test dataset size:  429450


In [15]:
train_dataset.head()

,titles,ingredients,directions
876994,Sausage And Cheese Log,[10 oz. grated sharp Cheddar cheese (room temp...,"[Mix all together; form into 4 to 6 logs., Wra..."
681739,"""Mexican Eggs""","[6 eggs, 2 Tbsp. melted butter, 1/4 c. flour, ...","[Whisk together first 4 ingredients., Add chil..."
1608519,Cheesy ham and potato casserole,"[1 packages county style shredded hash browns,...","[pre heat oven to 350F., Coat a 13*9 inch baki..."
1594363,Lamb Stew,"[3 pounds lamb stew meat cubed, 1/4 teaspoon t...",[Dry lamb cubes and brown in hot oil in Dutch ...
216121,Pickles,"[15 lb. cucumbers, sliced for pickles, 3/4 c. ...",[Soak cucumbers in solution of 3/4 cup salt to...


In [16]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [17]:
vectorizer = TfidfVectorizer()

ingredients_text = [' '.join(ingredient) for ingredient in train_dataset["ingredients"]]
X_ingredients = vectorizer.fit_transform(ingredients_text)

In [18]:
X_ingredients.shape

(1288348, 51125)

In [19]:
knn = NearestNeighbors(n_neighbors=1, metric='cosine')
knn.fit(X_ingredients)

NearestNeighbors(metric='cosine', n_neighbors=1)

In [20]:
def find_closest_recipe(ingredients_list):
    ingredients_text = ' '.join(ingredients_list)
    input_vector = vectorizer.transform([ingredients_text])

    # use the pre-fitted KNN model to find the closest recipe index
    closest_index = knn.kneighbors(input_vector, return_distance=False)[0][0]

    # get closest recipe details from the original dataset
    closest_recipe = train_dataset.iloc[closest_index]

    return {
        'title': closest_recipe['titles'],
        'ingredients': closest_recipe['ingredients'],
        'directions': closest_recipe['directions']
    }

In [22]:
# example input
input_ingredients = ['eggs', 'butter', 'flour']

closest_recipe = find_closest_recipe(input_ingredients)

print("Closest Recipe:")
print("Title:", closest_recipe['title'])
print("Ingredients:", closest_recipe['ingredients'])
print("Directions:", closest_recipe['directions'])

Closest Recipe:
Title: 1, 2, 3, 4 Cake
Ingredients: ['1 c. butter', '2 c. sugar', '3 c. flour', '4 eggs']
Directions: ['Try 1 teaspoon baking powder or try 1/2 teaspoon soda and 1 teaspoon cream of tartar.', 'You know how to put a cake together!']


In [25]:
import pickle

filename = 'recipe_knn_model.sav'
pickle.dump(knn, open(filename, 'wb'))

filename = 'knn_vectorizer.sav'
pickle.dump(vectorizer, open(filename, 'wb'))